# CIFAR10 Tensorflow Pytorch

This notebook walks you through image classification model training in Determined on the popular [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html), specifically using the PyTorch machine learning library.  See [this notebook](../cifar10_cnn_tf_keras/CIFAR10-Tensorflow-Keras.ipynb) for the same example built on TensorFlow Keras.

In [ ]:
# Test importing Determined.  In Determined is properly installed, you should see no output.
import determined as det

In [ ]:
# Replace with the IP address of the Determined master.
determined_master = '<master-ip>'

# Run an Experiment

First, we will explore the components of a Determined experiment; namely, the model definition and associated experiment configuration.

## Model Directory
- `model_def.py`: The PyTorch model definition
- `.yaml` configuration files that each govern an individual experiment run

Let's look at the contents of the model directory:

In [ ]:
!ls .

### model_def.py
Now drill in and view the model definition file.  Look for the implementation of Determined's `PyTorchTrial` interface.  This is the interface between Determined and PyTorch, which ultimately enables the ML Engineer to leverage Determined's distributed hyperparameter search in a shared runtime without having to worry about these distributed system concerns.

In [ ]:
!cat -n model_def.py

### const.yaml
For our first Determined experiment, we'll run this model training job with fixed hyperparameters.  Note the following sections:
- `description`: A short description of the experiment
- `data`: A section for user to provide custom key value pairs.  Here we specify where the data resides.  
- `hyperparameters`: area for user to define hyperparameters that will be injected into the trial class at runtime. There are constant values for this configuration
- `searcher`: hyperparameter search algorithm for the experiment

In [ ]:
!cat -n const.yaml

## Submit Experiment

In [ ]:
!det -m {determined_master} experiment create const.yaml .

Once the experiment completes (which may take a few minutes if Determined agents have to start up), look at the experiment page to see the single completed trial.  Note the validation error around 0.75.

# Adaptive Hyperparameter Search
### adaptive.yaml

Next, let's run an experiment with the same model definition, but we'll leverage Determined's adaptive hyperparameter search to efficiently determine the hyperparameter values that yield the lowest validation error.  Note that hyperparameters in the experiment configuration are specified as ranges as opposed to fixed values as in our [first experiment](#const.yaml).

In [ ]:
!cat -n adaptive.yaml

## Submit Experiment

In [ ]:
!det -m {determined_master} experiment create adaptive.yaml .

During and after the experiment run, you can view the best (lowest) validation error that Determined's adaptive search finds over time:

When the experiment finishes, note that your best performing model achieves a lower validation error than our first experiment that ran with constant hyperparameter values.  From the Determined experiment detail page, you can drill in to a particular trial and view the hyperparameter values used.  You can also access the saved checkpoint of your best-performing model and load it for real-time or batch inference as described in the PyTorch documentation [here](https://pytorch.org/tutorials/beginner/saving_loading_models.html#saving-loading-a-general-checkpoint-for-inference-and-or-resuming-training).